<a href="https://colab.research.google.com/github/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/blob/main/Project_HPE_Trail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
!wget https://github.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/blob/main/English_Danish_Dutch.xlsx?raw=true

--2021-01-27 22:36:10--  https://github.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/blob/main/English_Danish_Dutch.xlsx?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/raw/main/English_Danish_Dutch.xlsx [following]
--2021-01-27 22:36:11--  https://github.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/raw/main/English_Danish_Dutch.xlsx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/English_Danish_Dutch.xlsx [following]
--2021-01-27 22:36:11--  https://raw.githubusercontent.com/404S-retr0/Even_Dead_I_Am-_The_Hero-E.D.I.T.H-/main/English_Danish_Dutch.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.

In [2]:
df=pd.read_excel('/content/English_Danish_Dutch.xlsx?raw=true')

In [3]:
df

,words,Langauage_vector
0,the,1
1,to,1
2,and,1
3,a,1
4,of,1
...,...,...
257363,geurloze,3
257364,straftijd,3
257365,strafvordering,3
257366,onbekommerde,3


In [4]:
int2word = {}
word2int = {}
vocab_size = len(df.words)
for i,word in enumerate(df['words'].values):
  int2word[i] = df.words[i] 
  word2int[df.words[i]] = i

In [5]:
df['int2word'] = int2word

In [6]:
df

,words,Langauage_vector,int2word
0,the,1,0
1,to,1,1
2,and,1,2
3,a,1,3
4,of,1,4
...,...,...,...
257363,geurloze,3,257363
257364,straftijd,3,257364
257365,strafvordering,3,257365
257366,onbekommerde,3,257366


In [223]:
x,y=df['int2word'].values,df['Langauage_vector'].values
x=x.reshape(-1,1)
y=y.reshape(-1,1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(205894, 1)
(51474, 1)
(205894, 1)
(51474, 1)


In [82]:
x[300][0]

300

In [224]:
from sklearn.neighbors import KNeighborsClassifier
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3)
# Fit the classifier to the data
knn.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [225]:
knn.score(x_test, y_test)

0.9999805727163228

In [85]:
x_test[0:5]

array([[ 66450],
       [175930],
       [238928],
       [ 29102],
       [131947]])

In [86]:
knn.predict(x_test)[0:5]

array([1, 2, 3, 1, 2])

In [87]:
text1='hello my name is'

In [191]:
knn.predict(x)[word2int['my']]

1

In [245]:
def lang_detect_in_percentage(text):
  import string
  import re
  test_str=text
  test_str = ''.join([i for i in test_str if not i.isdigit()]) 
  test_str=test_str.lower()
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~+-=|`'''

  for ele in test_str:
    if ele in punc:
      test_str = test_str.replace(ele, "")  

  test_str_split=re.split('\s+', test_str)
  
  if (test_str_split[0]==''):
    test_str_split.remove('')
  if (test_str_split[len(test_str_split)-1]==''):
    test_str_split.remove('')
  

  eng=0
  dan=0
  dut=0
  unknown=0

  list_length = len(test_str_split)
  for q in range(list_length):
    try:
      vec=word2int[test_str_split[q]]
      pre=knn.predict(x)[vec]
    except:
      pre=-1
    else:
      if (pre==1):
        eng +=1
      elif (pre==2):
        dan +=1
      elif (pre==3):
        dut +=1
      else:
       unknown +=1
    
    if (pre==-1):
      unknown +=1


  p_eng=(eng/(eng+dan+dut+unknown))
  p_dan=(dan/(eng+dan+dut+unknown))
  p_dut=(dut/(eng+dan+dut+unknown))
  p_unk=(unknown/(eng+dan+dut+unknown))
  report = "Percentage Distribution\nEnglish:-{}\nDanish:-{}\nDutch:-{}\nUnknown:-{}".format( p_eng,p_dan,p_dut,p_unk)
  print(report)

In [246]:
lang_detect_in_percentage('hello my name is markeerde')

Percentage Distribution
English:-0.8
Danish:-0.0
Dutch:-0.2
Unknown:-0.0
